In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import torch
from torch import multiprocessing

In [3]:
is_fork = multiprocessing.get_start_method() == "fork"

device = (
    torch.device(0)
    if torch.cuda.is_available() and not is_fork
    else torch.device("cpu")
)

## Environment Preparation

#### Load unity environment using `mlagents_envs`

In [ ]:
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel
from mlagents_envs.environment import UnityEnvironment

channel = EngineConfigurationChannel()
env_path = "D:/_Thesis/warehouse-bot-training/environment_builds/stage1/warehouse_stage1_complex_pos_neg_3/Warehouse_Bot.exe"

unity_env = UnityEnvironment(
  file_name=env_path,
  side_channels=[channel],
  # additional_args=["-batchmode", "-nographics"]
)
channel.set_configuration_parameters(time_scale=5)

#### Transform environment from `mlagents` to `gymnasium`

In [ ]:
import gymnasium as gym
print(gym.__version__)

In [6]:
# from src.environments.env_camera_raycasts_gymnasium_wrapper import UnityCameraRaycastsGymWrapper
from src.environments.env_raycasts_gymnasium_wrapper import UnityRaycastsGymWrapper

gymnasium_env = UnityRaycastsGymWrapper(unity_env)

#### Creating stable_baselines3 model

In [ ]:
from stable_baselines3 import PPO

model_path = "./saved_models/baselines/stage1/warehouse_stage1_complex_pos_neg_3.6_12m"
model = PPO.load(model_path, gymnasium_env)

In [11]:
env = model.get_env()
obs = env.reset()

In [ ]:
for i in range(5000):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, info = env.step(action)